<a href="https://colab.research.google.com/github/cmfritz/capstone/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/Github/capstone

/content/gdrive/My Drive/Github/capstone


In [ ]:
#import packages
import pandas as pd
import numpy as np
import gc

In [ ]:
# Load data and take a first look
data = pd.read_csv('https://s3.amazonaws.com/data.patentsview.org/download/wipo_field.tsv.zip', 
                   compression='zip', sep='\t')
df_fields = pd.DataFrame(data)
df_fields

,id,sector_title,field_title
0,1,Electrical engineering,"Electrical machinery, apparatus, energy"
1,10,Instruments,Measurement
2,11,Instruments,Analysis of biological materials
3,12,Instruments,Control
4,13,Instruments,Medical technology
...,...,...,...
65,D30,Design,Articles for the care and handling of animals
66,D31,Design,Machines and appliances for preparing food or ...
67,D32,Design,"Graphic symbols and logos, surface patterns, o..."
68,D34,Design,Material or Article Handling Equipment


In [ ]:
df_fields = df_fields[df_fields.sector_title!=('Design' or 'Plant')]
df_fields.id = df_fields.id.astype('int64')
df_fields.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            36 non-null     int64 
 1   sector_title  36 non-null     object
 2   field_title   36 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.1+ KB


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df_fields.to_csv('data/fields.csv')

In [ ]:
# Load data and take a first look
data = pd.read_csv('https://s3.amazonaws.com/data.patentsview.org/download/wipo.tsv.zip', 
                   compression='zip', sep='\t')
df_field_lookup = pd.DataFrame(data)

In [ ]:
df_field_lookup = df_field_lookup.sort_values('patent_id')
df_field_lookup.to_csv('data/pats_w_sectors.csv')

In [ ]:
df_pats_w_sectors = df_field_lookup.merge(df_fields, 
                                          left_on='field_id', 
                                          right_on='id')
df_pats_w_sectors

,patent_id,field_id,sequence,id,sector_title,field_title
0,3930271,33,0,33,Other fields,"Furniture, games"
1,3930272,33,0,33,Other fields,"Furniture, games"
2,3930281,33,0,33,Other fields,"Furniture, games"
3,3930331,33,0,33,Other fields,"Furniture, games"
4,3930332,33,0,33,Other fields,"Furniture, games"
...,...,...,...,...,...,...
10296072,10818805,22,1,22,Chemistry,Micro-structural and nano-technology
10296073,10818835,22,2,22,Chemistry,Micro-structural and nano-technology
10296074,10818922,22,2,22,Chemistry,Micro-structural and nano-technology
10296075,10823691,22,3,22,Chemistry,Micro-structural and nano-technology


In [ ]:
df_pats_w_sectors = df_pats_w_sectors.drop(['field_id', 'sequence', 'id'], axis=1)
df_pats_w_sectors
df_pats_w_sectors.to_csv('data/pats_w_sectors.csv')

In [ ]:
len(set(df_pats_w_sectors.patent_id))

6836110

In [ ]:
# Load 2020 data
data = pd.read_csv('https://s3.amazonaws.com/data.patentsview.org/claims/claims_2020.tsv.zip',
                   compression='zip', sep='\t')
df = pd.DataFrame(data)
del data
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,uuid,patent_id,text,dependent,sequence,num,exemplary
0,87720c54-7d3e-11ea-83c3-121df0c29c1e,10524402,1. A method comprising: capturing an image of ...,NaN,0,1,1
1,87720dc3-7d3e-11ea-83c3-121df0c29c1e,10524402,"2. The method of claim 1 , wherein identifying...",claim 1,1,2,0
2,877214e9-7d3e-11ea-83c3-121df0c29c1e,10524402,"3. The method of claim 1 , wherein the propert...",claim 1,2,3,0
3,87721630-7d3e-11ea-83c3-121df0c29c1e,10524402,"4. The method of claim 1 , wherein selecting a...",claim 1,3,4,0
4,87721730-7d3e-11ea-83c3-121df0c29c1e,10524402,"5. The method of claim 1 , wherein the propert...",claim 1,4,5,0


In [ ]:
df = df.drop(['uuid', 'sequence', 'exemplary', 'dependent'], 
             axis=1)

# rename num column since num name causes issues
df = df.rename(columns={'num':'claim_number'})

# get rid of reissue, design, and plant patents
df = df[df.patent_id.str.isnumeric()==True]

# make sure dtype for patid is integer
df.patent_id = df.patent_id.astype('int64')

# keep only rows with 1st claim
df = df[df.claim_number==1]

df_master = df
del df
gc.collect()
df_master

,patent_id,text,claim_number
5767175,10873754,"1. A method for encoding video data, wherein t...",1
5767195,10873755,1. A computer-implemented method of motion est...,1
5767220,10873756,"1. A video processing method, comprising: cons...",1
5767245,10873757,1. A method of decoding video data in uni-dire...,1
5767254,10873758,1. An image processing device comprising: a se...,1
...,...,...,...
5888339,10881037,1. A method of manufacturing a printed circuit...,1
5888358,10881038,"1. A control system for a motor vehicle, in pa...",1
5888378,10881039,1. An Electromagnetic Interference shielding f...,1
5888398,10881040,"1. An adjustable mobile cart, including: at le...",1


In [ ]:
def clean_tsv(df):
  df = df.drop(['uuid', 'sequence', 'exemplary', 'dependent', 
              'patent_date','created_date', 'updated_date'], 
             axis=1)

  # get rid of reissue, design, and plant patents
  df = df[df.patent_id.str.isnumeric()==True]

  # make sure dtype for patid is integer
  df.patent_id = df.patent_id.astype('int64')

  # keep rows only for claim 1
  df = df[df.claim_number==1]

  return df

In [ ]:
# Iterate to load 2016 - 2019 data
years = range(2016,2020,1)
for year in years:
  data = pd.read_csv(f'https://s3.amazonaws.com/data.patentsview.org/claims/claims_{year}.tsv.zip',
                   compression='zip', sep='\t')
  df = pd.DataFrame(data)
  del data
  df = clean_tsv(df)
  df_master=df_master.append(df, ignore_index=True)
  del df
  gc.collect()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_master

,patent_id,text,claim_number
0,10873754,"1. A method for encoding video data, wherein t...",1
1,10873755,1. A computer-implemented method of motion est...,1
2,10873756,"1. A video processing method, comprising: cons...",1
3,10873757,1. A method of decoding video data in uni-dire...,1
4,10873758,1. An image processing device comprising: a se...,1
...,...,...,...
557406,10395357,1. An apparatus for detecting foam in a specim...,1
557407,10395358,1. A system configured to detect defects on a ...,1
557408,10395359,1. A system configured to detect defects on a ...,1
557409,10395360,1. An inspection system for inspecting an obje...,1
